In [2]:
# Cell 1: Setup and Initial Cohort Loading
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

def connect_db():
    """Establish database connection"""
    try:
        engine = create_engine('***')
        connection = engine.connect()
        print("Connected to mimiciv successfully!")
        return engine
    except Exception as e:
        print(f"Connection failed: {e}")
        return None

def load_validated_cohort(engine):
    """Load the validated CLABSI 2.0 cohort with all required features"""
    
    cohort_query = text("""
    WITH clabsi_icd AS (
        SELECT DISTINCT hadm_id, MIN(icd_code) as icd_code
        FROM mimiciv_hosp.diagnoses_icd
        WHERE icd_code IN ('99931', '99932', 'T80211A')
        GROUP BY hadm_id
    ),
    validated_cases AS (
        SELECT DISTINCT
            i.subject_id,
            i.hadm_id,
            i.stay_id,
            i.intime as icu_admission,
            i.outtime as icu_discharge,
            l.starttime as line_start,
            l.endtime as line_end,
            l.line_type,
            pc.charttime as infection_time,
            pc.org_name,
            pc.cultures_of_org,
            pc.daily_org_cultures,
            CASE
                WHEN pc.org_name IN (
                    'STAPH AUREUS COAG +', 'ESCHERICHIA COLI',
                    'KLEBSIELLA PNEUMONIAE', 'PSEUDOMONAS AERUGINOSA',
                    'ENTEROCOCCUS FAECIUM', 'CANDIDA ALBICANS'
                ) OR (
                    pc.org_name IN (
                        'STAPHYLOCOCCUS, COAGULASE NEGATIVE',
                        'STAPHYLOCOCCUS EPIDERMIDIS'
                    ) AND pc.daily_org_cultures >= 2
                )
                THEN 'Confirmed CLABSI'
                WHEN pc.daily_org_cultures >= 2
                THEN 'Possible CLABSI'
                ELSE 'Requires Review'
            END as clabsi_status
        FROM mimiciv_icu.icustays i
        INNER JOIN clabsi_icd c ON i.hadm_id = c.hadm_id
        INNER JOIN mimiciv_derived.invasive_line l ON i.stay_id = l.stay_id
        INNER JOIN (
            SELECT 
                hadm_id,
                charttime,
                org_name,
                COUNT(*) OVER (
                    PARTITION BY hadm_id, org_name
                ) as cultures_of_org,
                COUNT(*) OVER (
                    PARTITION BY hadm_id, 
                    DATE_TRUNC('day', charttime),
                    org_name
                ) as daily_org_cultures
            FROM mimiciv_hosp.microbiologyevents
            WHERE spec_type_desc = 'BLOOD CULTURE'
            AND org_name IS NOT NULL
        ) pc ON i.hadm_id = pc.hadm_id
        WHERE l.line_type IN (
            'PICC', 'Multi Lumen', 'Dialysis', 'Triple Introducer',
            'Pre-Sep', 'Hickman', 'Portacath', 'Cordis/Introducer',
            'Continuous Cardiac Output PA', 'PA'
        )
        AND pc.charttime > l.starttime
        AND pc.charttime <= l.starttime + INTERVAL '14 days'
        AND EXTRACT(EPOCH FROM (l.endtime - l.starttime))/86400 >= 2
    )
    SELECT *
    FROM validated_cases
    WHERE clabsi_status = 'Confirmed CLABSI'
    ORDER BY stay_id, infection_time;
    """)
    
    cohort_df = pd.read_sql(cohort_query, engine)
    print("\nCLABSI 2.0 Cohort Summary:")
    print("-" * 30)
    print(f"Total confirmed cases: {len(cohort_df)}")
    print(f"Unique patients: {cohort_df['subject_id'].nunique()}")
    print(f"Unique ICU stays: {cohort_df['stay_id'].nunique()}")
    print("\nLine Type Distribution:")
    print(cohort_df['line_type'].value_counts())
    return cohort_df

# Connect to database and load cohort
engine = connect_db()
if engine:
    clabsi_cohort = load_validated_cohort(engine)

Connected to mimiciv successfully!

CLABSI 2.0 Cohort Summary:
------------------------------
Total confirmed cases: 634
Unique patients: 141
Unique ICU stays: 158

Line Type Distribution:
line_type
Multi Lumen                     240
Dialysis                        183
PICC                            117
PA                               38
Cordis/Introducer                28
Hickman                          13
Portacath                        11
Continuous Cardiac Output PA      4
Name: count, dtype: int64


In [3]:
# Cell 2: Feature Engineering for Temporal and Clinical Risk Factors

def get_clinical_features(engine, cohort_df):
    """Extract clinical features in pre-infection window"""
    
    features_query = text("""
    WITH infection_times AS (
        -- Get earliest positive culture time for each validated case
        SELECT 
            stay_id,
            MIN(infection_time) as event_time
        FROM unnest(:stay_ids, :event_times) AS t(stay_id, infection_time)
        GROUP BY stay_id
    ),
    vitals_pre_infection AS (
        -- Get vital signs in 72h window before infection
        SELECT 
            v.stay_id,
            v.charttime,
            v.heart_rate,
            v.sbp,
            v.dbp,
            v.mbp,
            v.resp_rate,
            v.temperature,
            v.spo2
        FROM mimiciv_derived.vitalsign v
        INNER JOIN infection_times it ON v.stay_id = it.stay_id
        WHERE v.charttime <= it.event_time
        AND v.charttime >= it.event_time - INTERVAL '72 hours'
    ),
    labs_pre_infection AS (
        -- Get lab values in 72h window
        SELECT 
            ie.stay_id,
            MAX(CASE WHEN itemid = 51300 THEN valuenum END) as wbc,
            MAX(CASE WHEN itemid = 51265 THEN valuenum END) as platelet,
            MAX(CASE WHEN itemid = 50862 THEN valuenum END) as albumin,
            MAX(CASE WHEN itemid = 50912 THEN valuenum END) as creatinine
        FROM mimiciv_icu.icustays ie
        INNER JOIN infection_times it ON ie.stay_id = it.stay_id
        INNER JOIN mimiciv_hosp.labevents le ON ie.subject_id = le.subject_id
        WHERE le.charttime <= it.event_time
        AND le.charttime >= it.event_time - INTERVAL '72 hours'
        AND le.itemid IN (
            51300, -- WBC
            51265, -- Platelet Count
            50862, -- Albumin
            50912  -- Creatinine
        )
        GROUP BY ie.stay_id
    ),
    severity_pre_infection AS (
        -- Get severity scores
        SELECT 
            s.stay_id,
            MAX(s.sofa_24hours) as sofa,
            MAX(s.respiration_24hours) as respiration_score,
            MAX(s.coagulation_24hours) as coagulation_score,
            MAX(s.cardiovascular_24hours) as cardiovascular_score,
            MAX(s.cns_24hours) as cns_score,
            MAX(s.renal_24hours) as renal_score
        FROM mimiciv_derived.sofa s
        INNER JOIN infection_times it ON s.stay_id = it.stay_id
        WHERE s.starttime <= it.event_time
        AND s.starttime >= it.event_time - INTERVAL '72 hours'
        GROUP BY s.stay_id
    ),
    line_care AS (
        -- Get line care documentation
        SELECT 
            ce.stay_id,
            COUNT(CASE WHEN ce.itemid = 228137 THEN 1 END) as chg_baths,
            COUNT(CASE WHEN ce.itemid IN (227293, 227358, 227357) THEN 1 END) as dressing_changes,
            COUNT(CASE WHEN ce.itemid IN (224188, 224289) THEN 1 END) as site_assessments
        FROM mimiciv_icu.chartevents ce
        INNER JOIN infection_times it ON ce.stay_id = it.stay_id
        WHERE ce.itemid IN (
            228137,  -- CHG Bath
            227293,  -- Multi Lumen Dressing
            227358,  -- PICC Dressing
            227357,  -- Dialysis Cath Dressing
            224188,  -- PICC Site Assessment
            224289   -- Line Site Assessment
        )
        AND ce.charttime <= it.event_time
        AND ce.charttime >= it.event_time - INTERVAL '72 hours'
        GROUP BY ce.stay_id
    )
    -- Aggregate features for each case
    SELECT 
        v.stay_id,
        -- Vital Signs
        MIN(v.heart_rate) as hr_min_72h,
        MAX(v.heart_rate) as hr_max_72h,
        MIN(v.temperature) as temp_min_72h,
        MAX(v.temperature) as temp_max_72h,
        MIN(v.mbp) as mbp_min_72h,
        -- Labs
        MAX(l.wbc) as wbc_max_72h,
        MIN(l.wbc) as wbc_min_72h,
        MIN(l.platelet) as plt_min_72h,
        -- Severity
        MAX(s.sofa) as max_sofa_72h,
        MAX(s.respiration_score) as max_resp_score_72h,
        MAX(s.cardiovascular_score) as max_cv_score_72h,
        -- Line Care
        COALESCE(lc.chg_baths, 0) as chg_baths_72h,
        COALESCE(lc.dressing_changes, 0) as dressing_changes_72h,
        COALESCE(lc.site_assessments, 0) as site_assessments_72h
    FROM vitals_pre_infection v
    LEFT JOIN labs_pre_infection l ON v.stay_id = l.stay_id
    LEFT JOIN severity_pre_infection s ON v.stay_id = s.stay_id
    LEFT JOIN line_care lc ON v.stay_id = lc.stay_id
    GROUP BY v.stay_id, lc.chg_baths, lc.dressing_changes, lc.site_assessments;
    """)
    
    # Prepare parameters
    stay_ids = cohort_df['stay_id'].tolist()
    event_times = cohort_df['infection_time'].tolist()
    
    # Execute query
    features_df = pd.read_sql(
        features_query, 
        engine,
        params={'stay_ids': stay_ids, 'event_times': event_times}
    )
    
    print("\nFeature Engineering Summary:")
    print("-" * 30)
    print(f"Features extracted for {len(features_df)} cases")
    print("\nFeature Statistics:")
    print(features_df.describe().round(2))
    return features_df

if 'clabsi_cohort' in locals():
    clinical_features = get_clinical_features(engine, clabsi_cohort)


Feature Engineering Summary:
------------------------------
Features extracted for 139 cases

Feature Statistics:
           stay_id  hr_min_72h  hr_max_72h  temp_min_72h  temp_max_72h  \
count       139.00      139.00      139.00        138.00        138.00   
mean   35281173.53       75.78      116.60         36.37         37.97   
std     2811331.33       18.34       21.38          0.69          0.99   
min    30015010.00       40.00       65.00         33.28         34.39   
25%    32797116.50       62.00      101.00         36.02         37.22   
50%    35692039.00       72.00      117.00         36.50         38.00   
75%    37588530.00       86.00      131.50         36.76         38.61   
max    39859332.00      135.00      171.00         38.17         40.11   

       mbp_min_72h  wbc_max_72h  wbc_min_72h  plt_min_72h  max_sofa_72h  \
count       139.00          1.0          1.0       139.00        139.00   
mean         55.32          4.8          4.8       242.26          7

In [4]:
# Cell 3: Demographic and Admission Features

def get_demographic_features(engine, cohort_df):
    """Extract demographic and admission-related features"""
    
    demo_query = text("""
    WITH admission_features AS (
        -- Get admission and demographic information
        SELECT 
            icu.stay_id,
            icu.hadm_id,
            icu.subject_id,
            -- Demographics
            p.gender,
            p.anchor_age + EXTRACT(EPOCH FROM adm.admittime - MAKE_TIMESTAMP(p.anchor_year, 1, 1, 0, 0, 0))/31556908.8 AS admission_age,
            adm.race,
            adm.language,
            adm.insurance,
            adm.marital_status,
            -- Admission details
            adm.admission_type,
            adm.admission_location,
            -- Service and location
            EXTRACT(EPOCH FROM (icu.outtime - icu.intime))/86400 as icu_los_days,
            -- Transfer info
            CASE WHEN previous.stay_id IS NOT NULL THEN 1 ELSE 0 END as previous_icu_stay,
            ROW_NUMBER() OVER (PARTITION BY icu.hadm_id ORDER BY icu.intime) as icu_stay_number,
            -- Time of day/week features
            EXTRACT(HOUR FROM icu.intime) as admission_hour,
            EXTRACT(DOW FROM icu.intime) as admission_dow,
            -- Elective vs urgent
            CASE WHEN adm.admission_type = 'ELECTIVE' THEN 1 ELSE 0 END as is_elective
        FROM mimiciv_icu.icustays icu
        INNER JOIN mimiciv_hosp.admissions adm 
            ON icu.hadm_id = adm.hadm_id
        INNER JOIN mimiciv_hosp.patients p 
            ON icu.subject_id = p.subject_id
        -- Self-join to find previous ICU stays
        LEFT JOIN mimiciv_icu.icustays previous 
            ON icu.subject_id = previous.subject_id
            AND icu.intime > previous.outtime
            AND icu.hadm_id != previous.hadm_id
        WHERE icu.stay_id IN :stay_ids
    ),
    charlson_scores AS (
        -- Get Charlson comorbidity information
        SELECT
            icu.stay_id,
            MAX(ch.myocardial_infarct) as has_mi,
            MAX(ch.congestive_heart_failure) as has_chf,
            MAX(ch.peripheral_vascular_disease) as has_pvd,
            MAX(ch.cerebrovascular_disease) as has_stroke,
            MAX(ch.chronic_pulmonary_disease) as has_copd,
            MAX(ch.rheumatic_disease) as has_rheumatic,
            MAX(ch.mild_liver_disease) as has_mild_liver,
            MAX(ch.severe_liver_disease) as has_severe_liver,
            MAX(ch.diabetes_without_cc) as has_diabetes,
            MAX(ch.diabetes_with_cc) as has_diabetes_complicated,
            MAX(ch.paraplegia) as has_paralysis,
            MAX(ch.renal_disease) as has_renal,
            MAX(ch.malignant_cancer) as has_malignancy,
            MAX(ch.metastatic_solid_tumor) as has_metastasis,
            MAX(ch.aids) as has_aids,
            MAX(ch.charlson_comorbidity_index) as charlson_score
        FROM mimiciv_icu.icustays icu
        LEFT JOIN mimiciv_derived.charlson ch
            ON icu.subject_id = ch.subject_id
            AND icu.hadm_id = ch.hadm_id
        WHERE icu.stay_id IN :stay_ids
        GROUP BY icu.stay_id
    ),
    surgery_info AS (
        -- Get surgical procedures during admission
        SELECT 
            icu.stay_id,
            COUNT(DISTINCT pr.chartdate) as surgery_count,
            MAX(CASE 
                WHEN LOWER(d.long_title) LIKE '%%emergency%%' 
                OR LOWER(d.long_title) LIKE '%%urgent%%' 
                THEN 1 
                ELSE 0 
            END) as had_emergency_surgery
        FROM mimiciv_icu.icustays icu
        LEFT JOIN mimiciv_hosp.procedures_icd pr
            ON icu.hadm_id = pr.hadm_id
            AND pr.chartdate <= icu.outtime
        LEFT JOIN mimiciv_hosp.d_icd_procedures d
            ON pr.icd_code = d.icd_code 
            AND pr.icd_version = d.icd_version
        WHERE icu.stay_id IN :stay_ids
        AND (
            pr.icd_code LIKE '0%%' OR  -- ICD-10 procedures
            LOWER(d.long_title) LIKE '%%surgery%%' OR
            LOWER(d.long_title) LIKE '%%operation%%'
        )
        GROUP BY icu.stay_id
    )
    SELECT 
        af.*,
        -- Comorbidities (0/1 indicators)
        COALESCE(ch.has_mi, 0) as has_mi,
        COALESCE(ch.has_chf, 0) as has_chf,
        COALESCE(ch.has_pvd, 0) as has_pvd,
        COALESCE(ch.has_stroke, 0) as has_stroke,
        COALESCE(ch.has_copd, 0) as has_copd,
        COALESCE(ch.has_rheumatic, 0) as has_rheumatic,
        COALESCE(ch.has_mild_liver, 0) as has_mild_liver,
        COALESCE(ch.has_severe_liver, 0) as has_severe_liver,
        COALESCE(ch.has_diabetes, 0) as has_diabetes,
        COALESCE(ch.has_diabetes_complicated, 0) as has_diabetes_complicated,
        COALESCE(ch.has_paralysis, 0) as has_paralysis,
        COALESCE(ch.has_renal, 0) as has_renal,
        COALESCE(ch.has_malignancy, 0) as has_malignancy,
        COALESCE(ch.has_metastasis, 0) as has_metastasis,
        COALESCE(ch.has_aids, 0) as has_aids,
        COALESCE(ch.charlson_score, 0) as charlson_score,
        -- Surgery information
        COALESCE(s.surgery_count, 0) as surgery_count,
        COALESCE(s.had_emergency_surgery, 0) as had_emergency_surgery
    FROM admission_features af
    LEFT JOIN charlson_scores ch ON af.stay_id = ch.stay_id
    LEFT JOIN surgery_info s ON af.stay_id = s.stay_id;
    """)
    
    # Execute query
    demo_df = pd.read_sql(
        demo_query, 
        engine,
        params={'stay_ids': tuple(cohort_df['stay_id'].tolist())}
    )
    
    # Create binary features from categorical variables
    categorical_cols = ['admission_type', 'admission_location', 'race', 'insurance', 'marital_status']
    demo_df_encoded = pd.get_dummies(demo_df, columns=categorical_cols, prefix=categorical_cols)
    
    print("\nDemographic Features Summary:")
    print("-" * 30)
    print(f"Features extracted for {len(demo_df)} cases")
    print("\nNumerical Feature Statistics:")
    numerical_cols = ['admission_age', 'icu_los_days', 'charlson_score', 'surgery_count']
    print(demo_df[numerical_cols].describe().round(2))
    print("\nComorbidity Frequencies:")
    comorbidity_cols = [col for col in demo_df.columns if col.startswith('has_')]
    print(demo_df[comorbidity_cols].sum().sort_values(ascending=False))
    
    return demo_df_encoded

if 'clabsi_cohort' in locals():
    demographic_features = get_demographic_features(engine, clabsi_cohort)


Demographic Features Summary:
------------------------------
Features extracted for 257 cases

Numerical Feature Statistics:
       admission_age  icu_los_days  charlson_score  surgery_count
count         257.00        257.00          257.00         257.00
mean           59.10         15.08            5.45           1.87
std            15.82         15.91            2.91           2.90
min            24.25          1.03            0.00           0.00
25%            47.47          4.24            3.00           0.00
50%            64.47          9.46            6.00           1.00
75%            69.82         20.51            7.00           2.00
max            91.23         91.01           17.00          20.00

Comorbidity Frequencies:
has_renal                   121
has_chf                     102
has_copd                     81
has_diabetes                 60
has_mi                       57
has_stroke                   45
has_malignancy               44
has_diabetes_complicated     4

In [5]:
def get_line_care(engine, cohort_df):
    """Extract line care documentation"""
    
    line_care_query = text("""
    WITH line_care AS (
        SELECT 
            ce.stay_id,
            ce.charttime,
            CASE 
                WHEN ce.itemid = 228137 THEN 'CHG_BATH'
                WHEN ce.itemid IN (227293, 227358, 227357, 227359) THEN 'DRESSING_CHANGE'
                WHEN ce.itemid IN (224188, 224289) THEN 'SITE_ASSESSMENT'
            END as care_type,
            ce.valuenum,
            ce.value
        FROM mimiciv_icu.chartevents ce
        WHERE ce.stay_id IN :stay_ids
        AND ce.itemid IN (
            228137,  -- CHG Bath
            227293,  -- Multi Lumen Dressing
            227358,  -- PICC Dressing
            227357,  -- Dialysis Cath Dressing
            227359,  -- Tunneled Dressing
            224188,  -- PICC Site Assessment
            224289   -- Line Site Assessment
        )
    )
    SELECT 
        stay_id,
        COUNT(CASE WHEN care_type = 'CHG_BATH' THEN 1 END) as chg_bath_count,
        COUNT(CASE WHEN care_type = 'DRESSING_CHANGE' THEN 1 END) as dressing_changes,
        COUNT(CASE WHEN care_type = 'SITE_ASSESSMENT' THEN 1 END) as site_assessments
    FROM line_care
    GROUP BY stay_id
    """)
    
    # Convert numpy int64 to regular Python integers
    stay_ids = tuple(int(x) for x in cohort_df['stay_id'].unique())
    return pd.read_sql(line_care_query, 
                      engine,
                      params={'stay_ids': stay_ids})

In [6]:
def get_lab_values(engine, cohort_df):
    """Extract relevant laboratory values"""
    
    labs_query = text("""
    SELECT
        ie.stay_id,
        le.subject_id,
        le.charttime,
        MAX(CASE WHEN itemid = 51300 THEN valuenum END) as wbc,
        MAX(CASE WHEN itemid = 51301 THEN valuenum END) as wbc_corrected,
        MAX(CASE WHEN itemid = 51221 THEN valuenum END) as hematocrit,
        MAX(CASE WHEN itemid = 51222 THEN valuenum END) as hemoglobin,
        MAX(CASE WHEN itemid = 51265 THEN valuenum END) as platelet,
        MAX(CASE WHEN itemid = 50862 THEN valuenum END) as albumin,
        MAX(CASE WHEN itemid = 50912 THEN valuenum END) as creatinine
    FROM mimiciv_icu.icustays ie
    LEFT JOIN mimiciv_hosp.labevents le
        ON ie.subject_id = le.subject_id
        AND le.charttime >= ie.intime
        AND le.charttime <= ie.outtime
    WHERE ie.stay_id IN :stay_ids
    AND le.itemid IN (
        51300, -- WBC
        51301, -- WBC Corrected
        51221, -- Hematocrit
        51222, -- Hemoglobin
        51265, -- Platelet Count
        50862, -- Albumin
        50912  -- Creatinine
    )
    GROUP BY ie.stay_id, le.subject_id, le.charttime
    ORDER BY ie.stay_id, le.charttime
    """)
    
    # Convert numpy int64 to regular Python integers
    stay_ids = tuple(int(x) for x in cohort_df['stay_id'].unique())
    return pd.read_sql(labs_query, 
                      engine,
                      params={'stay_ids': stay_ids})

In [7]:
def get_severity_scores(engine, cohort_df):
    """Calculate severity scores including SOFA"""
    
    severity_query = text("""
    WITH sofa AS (
        -- Get SOFA scores from derived table
        SELECT 
            stay_id,
            MAX(sofa_24hours) as max_sofa,
            AVG(sofa_24hours) as avg_sofa,
            MAX(respiration_24hours) as resp_sofa,
            MAX(coagulation_24hours) as coag_sofa,
            MAX(liver_24hours) as liver_sofa,
            MAX(cardiovascular_24hours) as cv_sofa,
            MAX(cns_24hours) as cns_sofa,
            MAX(renal_24hours) as renal_sofa
        FROM mimiciv_derived.sofa
        WHERE stay_id IN :stay_ids
        GROUP BY stay_id
    ),
    vs AS (
        -- First day vital signs
        SELECT 
            ce.stay_id,
            MIN(vs.mbp) AS mbp_min,
            MIN(vs.heart_rate) as heart_rate_min,
            MAX(vs.heart_rate) as heart_rate_max,
            MIN(vs.resp_rate) as resp_rate_min,
            MAX(vs.resp_rate) as resp_rate_max,
            MIN(vs.temperature) as temperature_min,
            MAX(vs.temperature) as temperature_max
        FROM mimiciv_icu.icustays ce
        LEFT JOIN mimiciv_derived.vitalsign vs
            ON ce.stay_id = vs.stay_id
            AND vs.charttime >= ce.intime
            AND vs.charttime <= ce.intime + interval '24 hours'
        WHERE ce.stay_id IN :stay_ids
        GROUP BY ce.stay_id
    )
    SELECT 
        v.stay_id,
        s.max_sofa,
        s.avg_sofa,
        s.resp_sofa,
        s.coag_sofa,
        s.liver_sofa,
        s.cv_sofa,
        s.cns_sofa,
        s.renal_sofa,
        v.mbp_min,
        v.heart_rate_max,
        v.resp_rate_max,
        v.temperature_max
    FROM vs v
    LEFT JOIN sofa s ON v.stay_id = s.stay_id
    ORDER BY v.stay_id;
    """)
    
    stay_ids = tuple(int(x) for x in cohort_df['stay_id'].unique())
    return pd.read_sql(severity_query, engine, params={'stay_ids': stay_ids})

In [8]:
def get_medications(engine, cohort_df):
    """Extract medication features focusing on high-risk medications"""
    
    med_query = text("""
    WITH antibiotics AS (
        -- Get antibiotic exposure
        SELECT 
            stay_id,
            COUNT(DISTINCT antibiotic) as distinct_antibiotics,
            COUNT(*) as total_antibiotic_orders,
            MAX(EXTRACT(EPOCH FROM (stoptime - starttime))/86400) as max_antibiotic_duration
        FROM mimiciv_derived.antibiotic
        WHERE stay_id IN :stay_ids
        GROUP BY stay_id
    ),
    tpn AS (
        -- Total Parenteral Nutrition
        SELECT 
            stay_id,
            COUNT(*) as tpn_orders,
            SUM(CASE WHEN rate > 0 THEN 1 ELSE 0 END) as tpn_days
        FROM mimiciv_icu.inputevents
        WHERE stay_id IN :stay_ids
        AND itemid IN (
            226089, -- TPN
            227690  -- Lipids
        )
        GROUP BY stay_id
    ),
    high_risk_meds AS (
        -- Other high-risk medications
        SELECT 
            stay_id,
            COUNT(CASE WHEN itemid IN (
                221662, -- Vasopressors
                221653, -- Steroids
                221668  -- Immunosuppressants
            ) THEN 1 END) as high_risk_med_count,
            COUNT(DISTINCT itemid) as distinct_high_risk_meds
        FROM mimiciv_icu.inputevents
        WHERE stay_id IN :stay_ids
        GROUP BY stay_id
    )
    SELECT 
        COALESCE(a.stay_id, t.stay_id, h.stay_id) as stay_id,
        COALESCE(distinct_antibiotics, 0) as distinct_antibiotics,
        COALESCE(total_antibiotic_orders, 0) as total_antibiotic_orders,
        COALESCE(max_antibiotic_duration, 0) as max_antibiotic_duration,
        COALESCE(tpn_orders, 0) as tpn_orders,
        COALESCE(tpn_days, 0) as tpn_days,
        COALESCE(high_risk_med_count, 0) as high_risk_med_count,
        COALESCE(distinct_high_risk_meds, 0) as distinct_high_risk_meds
    FROM antibiotics a
    FULL OUTER JOIN tpn t ON a.stay_id = t.stay_id
    FULL OUTER JOIN high_risk_meds h ON a.stay_id = h.stay_id
    """)
    
    # Convert numpy int64 to regular Python integers
    stay_ids = tuple(int(x) for x in cohort_df['stay_id'].unique())
    return pd.read_sql(med_query, 
                      engine,
                      params={'stay_ids': stay_ids})

In [9]:
def get_time_windows(engine, cohort_df):
    """Extract events and metrics in pre-CLABSI window periods"""
    
    time_window_query = text("""
    WITH clabsi_events AS (
        SELECT i.stay_id, i.hadm_id, i.intime, i.outtime
        FROM mimiciv_icu.icustays i
        WHERE i.stay_id IN :stay_ids
    ),
    -- Get interventions 72h before
    interventions AS (
        SELECT 
            ce.stay_id,
            ce.charttime,
            di.label AS intervention_type,
            COUNT(*) OVER (
                PARTITION BY ce.stay_id, di.label
                ORDER BY ce.charttime 
                RANGE BETWEEN '72 HOURS' PRECEDING AND CURRENT ROW
            ) as interventions_72h
        FROM clabsi_events cl
        JOIN mimiciv_icu.chartevents ce ON cl.stay_id = ce.stay_id
        JOIN mimiciv_icu.d_items di ON ce.itemid = di.itemid
        WHERE di.label IN (
            'CHG Bath',
            'Dressing Change', 
            'Line Site Assessment'
        )
    ),
    -- Lab trends
    labs_72h AS (
        SELECT 
            ie.stay_id,
            AVG(CASE WHEN itemid = 51300 THEN valuenum END) as wbc_mean_72h,
            MAX(CASE WHEN itemid = 51300 THEN valuenum END) as wbc_max_72h,
            MIN(CASE WHEN itemid = 51265 THEN valuenum END) as platelet_min_72h
        FROM clabsi_events ie
        JOIN mimiciv_hosp.labevents le ON ie.hadm_id = le.hadm_id
        WHERE le.itemid IN (51300, 51265)  -- WBC, Platelets
        GROUP BY ie.stay_id
    )
    SELECT 
        i.stay_id,
        MAX(i.interventions_72h) as total_interventions_72h,
        l.wbc_mean_72h,
        l.wbc_max_72h,
        l.platelet_min_72h
    FROM interventions i
    LEFT JOIN labs_72h l ON i.stay_id = l.stay_id
    GROUP BY i.stay_id, l.wbc_mean_72h, l.wbc_max_72h, l.platelet_min_72h
    """)
    
    stay_ids = tuple(int(x) for x in cohort_df['stay_id'].unique())
    return pd.read_sql(time_window_query, 
                      engine,
                      params={'stay_ids': stay_ids})

In [10]:
def get_line_durations(engine, cohort_df):
    """Calculate line duration patterns and metrics"""
    
    duration_query = text("""
    WITH line_data AS (
        SELECT stay_id, line_type, starttime, endtime,
               EXTRACT(EPOCH FROM (endtime - starttime))/86400 as line_duration_days
        FROM mimiciv_derived.invasive_line
        WHERE line_type IN ('PICC', 'CVC', 'Tunneled')
          AND endtime > starttime
    ),
    line_counts AS (
        SELECT 
            stay_id,
            line_type,
            COUNT(*) as total_lines,
            SUM(line_duration_days) as total_duration_days,
            AVG(line_duration_days) as avg_duration_days,
            COUNT(*) OVER (PARTITION BY stay_id) as concurrent_lines
        FROM line_data
        GROUP BY stay_id, line_type
    )
    SELECT *
    FROM line_counts
    WHERE stay_id IN :stay_ids
    ORDER BY stay_id, total_duration_days DESC;
    """)
    
    stay_ids = tuple(int(x) for x in cohort_df['stay_id'].unique())
    return pd.read_sql(duration_query, engine, params={'stay_ids': stay_ids})

In [11]:
def get_dressing_care(engine, cohort_df):
    """Extract dressing care patterns and compliance metrics
    
    Returns:
    - detailed_events: All dressing assessments and changes
    - summary_metrics: Aggregated compliance and timing metrics
    """
    
    dressing_query = text("""
    WITH dressing_events AS (
        SELECT 
            ce.stay_id,
            ce.charttime,
            di.label,
            ce.value,
            CASE 
                WHEN ce.value = '1' AND di.label LIKE '%Occlusive%' THEN 'Intact'
                WHEN ce.value = '0' AND di.label LIKE '%Occlusive%' THEN 'Not Intact'
                WHEN ce.value = 'WNL' THEN 'Intact'
                WHEN ce.value IN ('Changed', 'Reinforced') THEN 'Changed'
                WHEN ce.value = 'Dry and intact' THEN 'Intact'
                WHEN ce.value IN ('Bleeding', 'Oozing', 'Old blood', 'Ecchymotic', 'Redness') THEN 'Concerning'
                WHEN ce.value = 'Not applicable' THEN 'Not Assessed'
                ELSE ce.value
            END as dressing_status,
            LAG(charttime) OVER (
                PARTITION BY ce.stay_id, di.label 
                ORDER BY charttime
            ) as prev_assessment
        FROM mimiciv_icu.chartevents ce
        JOIN mimiciv_icu.d_items di ON ce.itemid = di.itemid
        WHERE ce.itemid IN (
            224188,  -- PICC Line Site Appear
            227358,  -- PICC Line Dressing Occlusive
            224289,  -- Arterial line Site Appear
            227292,  -- Arterial Line Dressing
            227293,  -- Multi Lumen Dressing
            225323,  -- Dialysis Catheter Site Appear
            227357   -- Dialysis Catheter Dressing
        )
        AND ce.stay_id IN :stay_ids
    )
    SELECT 
        stay_id,
        label,
        charttime,
        value as original_value,
        dressing_status,
        EXTRACT(EPOCH FROM (charttime - prev_assessment))/3600 as hours_since_last_assessment,
        COUNT(*) OVER (
            PARTITION BY stay_id, DATE_TRUNC('day', charttime)
        ) as assessments_per_day
    FROM dressing_events
    ORDER BY stay_id, charttime;
    """)
    
    # Convert stay_ids to tuple
    stay_ids = tuple(int(x) for x in cohort_df['stay_id'].unique())
    
    # Get detailed events
    detailed_events = pd.read_sql(dressing_query, engine, params={'stay_ids': stay_ids})
    
    # Calculate summary metrics
    summary_metrics = {
        'status_distribution': detailed_events['dressing_status'].value_counts(),
        'mean_hours_between_assessments': detailed_events.groupby('label')['hours_since_last_assessment'].mean(),
        'daily_assessment_avg': detailed_events['assessments_per_day'].mean(),
        'concerning_dressings_pct': (detailed_events['dressing_status'] == 'Concerning').mean() * 100
    }
    
    return detailed_events, summary_metrics

In [12]:
def get_chg_bath_features(engine, cohort_df):
    """Extract CHG bath compliance and timing features"""
    
    bath_query = text("""
    WITH clabsi_timing AS (
        SELECT 
            i.stay_id,
            i.intime,
            m.charttime as infection_time,
            m.org_name
        FROM mimiciv_hosp.diagnoses_icd d
        JOIN mimiciv_icu.icustays i ON d.hadm_id = i.hadm_id
        JOIN mimiciv_hosp.microbiologyevents m ON d.hadm_id = m.hadm_id
        WHERE d.icd_code IN ('99931', '99932', 'T80211A')
        AND m.spec_type_desc = 'BLOOD CULTURE'
        AND m.org_name IS NOT NULL
    ),
    bath_timing AS (
        SELECT 
            ce.stay_id,
            ce.charttime as bath_time,
            ct.infection_time,
            ct.intime,
            EXTRACT(EPOCH FROM (ce.charttime - ct.infection_time))/3600 as hours_from_infection,
            EXTRACT(EPOCH FROM (ce.charttime - ct.intime))/3600 as hours_from_admission,
            LAG(ce.charttime) OVER (PARTITION BY ce.stay_id ORDER BY ce.charttime) as prev_bath
        FROM mimiciv_icu.chartevents ce
        JOIN clabsi_timing ct ON ce.stay_id = ct.stay_id
        WHERE ce.itemid = 228137  -- CHG Bath
    )
    SELECT 
        stay_id,
        COUNT(*) as total_baths,
        COUNT(CASE WHEN hours_from_infection < 0 THEN 1 END) as baths_before_infection,
        MIN(CASE WHEN hours_from_infection < 0 THEN ABS(hours_from_infection) END) as hours_since_last_bath,
        AVG(EXTRACT(EPOCH FROM (bath_time - prev_bath))/3600) as avg_hours_between_baths,
        COUNT(CASE 
            WHEN EXTRACT(EPOCH FROM (bath_time - prev_bath))/3600 > 24 
            THEN 1 
        END) as gaps_over_24h
    FROM bath_timing
    GROUP BY stay_id
    ORDER BY total_baths DESC;
    """)
    
    return pd.read_sql(bath_query, engine)

In [13]:
# Create database connection
engine = connect_db()

if engine is not None:
   print("Loading CLABSI cohort...")
   clabsi_df = load_validated_cohort(engine)
   print(f"Found {len(clabsi_df)} CLABSI cases")
   
   print("\nExtracting all features...")
   demographics = get_demographic_features(engine, clabsi_df)
   line_care = get_line_care(engine, clabsi_df)
   labs = get_lab_values(engine, clabsi_df)
   severity = get_severity_scores(engine, clabsi_df)
   medications = get_medications(engine, clabsi_df)
   time_windows = get_time_windows(engine, clabsi_df)
   line_durations = get_line_durations(engine, clabsi_df)
   dressing_events, dressing_summary = get_dressing_care(engine, clabsi_df)
   chg_baths = get_chg_bath_features(engine, clabsi_df)
   
   print("\nFeature extraction complete!")
   print(f"Demographics: {len(demographics)} rows")
   print(f"Line Care: {len(line_care)} rows")
   print(f"Lab Values: {len(labs)} rows")
   print(f"Severity Scores: {len(severity)} rows")
   print(f"Medications: {len(medications)} rows") 
   print(f"Time Windows: {len(time_windows)} rows")
   print(f"Line Durations: {len(line_durations)} rows")
   print(f"Dressing Events: {len(dressing_events)} rows")
   print(f"CHG Baths: {len(chg_baths)} rows")
   
   print("\nSOFA Score Statistics:")
   print(severity[['max_sofa', 'avg_sofa', 'resp_sofa', 'coag_sofa', 'liver_sofa', 'cv_sofa', 'cns_sofa', 'renal_sofa']].describe())

Connected to mimiciv successfully!
Loading CLABSI cohort...

CLABSI 2.0 Cohort Summary:
------------------------------
Total confirmed cases: 634
Unique patients: 141
Unique ICU stays: 158

Line Type Distribution:
line_type
Multi Lumen                     240
Dialysis                        183
PICC                            117
PA                               38
Cordis/Introducer                28
Hickman                          13
Portacath                        11
Continuous Cardiac Output PA      4
Name: count, dtype: int64
Found 634 CLABSI cases

Extracting all features...

Demographic Features Summary:
------------------------------
Features extracted for 257 cases

Numerical Feature Statistics:
       admission_age  icu_los_days  charlson_score  surgery_count
count         257.00        257.00          257.00         257.00
mean           59.10         15.08            5.45           1.87
std            15.82         15.91            2.91           2.90
min            24.25 